In [53]:
import pandas as pd

file = "../data/raw/pbh_data.xlsx"
sheet_names = pd.ExcelFile(file).sheet_names
print(sheet_names)

['boletins', 'envolvidos', 'veiculos', 'logradouros', 'coordenadas']


In [54]:
sheets = pd.read_excel(file, sheet_name=None)

for sheet in sheet_names:
    sheet = sheets[f"{sheet}"]
    print(f"Observations: {sheet.shape[0]}, Variables: {sheet.shape[1]}")
    display(sheet.head(2))

Observations: 71464, Variables: 21


,numero_boletim,tipo_acidente,desc_tipo_acidente,cod_tempo,desc_tempo,cod_pavimento,pavimento,cod_regional,desc_regional,origem_boletim,...,indicador_fatalidade,valor_ups,descricao_ups,Data,Hora,Faixa_hora,Faixa_hora_num,Mês,Ano,Dia
0,2019026821139001,H01002,ABALROAMENTO COM VITIMA,0,NAO INFORMADO,0,NAO INFORMADO,25,PAMPULHA,POLICIA MILITAR,...,NÃO,0,NÃO INFORMADO,2019-06-07,13:38:00,Tarde,2,6,2019,7
1,2019027020851001,H01002,ABALROAMENTO COM VITIMA,0,NAO INFORMADO,0,NAO INFORMADO,19,CENTRO-SUL,POLICIA MILITAR,...,NÃO,0,NÃO INFORMADO,2019-06-08,17:07:00,Tarde,2,6,2019,8


Observations: 154713, Variables: 16


,num_boletim,numero_envolvido,condutor,codigo_severidade,desc_severidade,sexo,cinto_seguranca,Embreagues,Idade,categoria_habilitacao,descricao_habilitacao,especie_veiculo,pedestre,passageiro,Data,Hora
0,2017004467915001,1,S,3,SEM FERIMENTOS,M,SIM,NÃO,25.0,B,"VEICULOS MOTOR. COM ATE 8 PASSAGEIROS, PESO MA...",AUTOMOVEL,N,N,2017-02-28,21:13:00
1,2017029493511001,1,S,3,SEM FERIMENTOS,M,SIM,NÃO,25.0,B,"VEICULOS MOTOR. COM ATE 8 PASSAGEIROS, PESO MA...",AUTOMOVEL,N,N,2017-10-10,13:31:00


Observations: 129826, Variables: 12


,Nº_boletim,seq_veic,cod_categ,descricao_categoria,cod_especie,descricao_especie,cod_situacao,desc_situacao,tipo_socorro,desc_tipo_socorro,Data,Hora
0,2016026389227001,1,3,PARTICULAR,6,AUTOMOVEL,1,EM MOVIMENTO,0,NÃO INFORMADO,2016-12-06,10:55:00
1,2016026413610001,1,3,PARTICULAR,6,AUTOMOVEL,1,EM MOVIMENTO,0,NÃO INFORMADO,2016-12-06,14:31:00


Observations: 93911, Variables: 11


,Nº_boletim,nome_municipio,seq_logradouros,tipo_logradouro,nome_logradouro,tipo_bairro,descricao_tipo_bairro,Nº_imovel,Bairro,Data,Hora
0,2016018874465001,BELO HORIZONTE,2,RUA,SAO PAULO,B,BAIRRO,0,Centro,2016-08-30,09:45:00
1,2016023140518001,BELO HORIZONTE,2,RUA,SAO PAULO,B,BAIRRO,0,Centro,2016-10-23,21:08:00


Observations: 73169, Variables: 3


,numero_bol,longitude,latitude
0,10017350,-44.013049,-19.990172
1,10078358,-43.898240,-19.862572


In [55]:
boletins.rename(columns={"numero_boletim": "numero_boletim"}, inplace=True)
envolvidos.rename(columns={"num_boletim": "numero_boletim"}, inplace=True)
veiculos.rename(columns={"Nº_boletim": "numero_boletim"}, inplace=True)
logradouros.rename(columns={"Nº_boletim": "numero_boletim"}, inplace=True)
coordenadas.rename(columns={"numero_bol": "numero_boletim"}, inplace=True)

In [56]:
for df in [boletins, envolvidos, veiculos, logradouros, coordenadas]:
    df["numero_boletim"] = df["numero_boletim"].astype(str).str.strip()

In [57]:
df = boletins.merge(envolvidos, on="numero_boletim", how="left", suffixes=("", "_envolvido")) \
                 .merge(veiculos, on="numero_boletim", how="left", suffixes=("", "_veiculo")) \
                 .merge(logradouros, on="numero_boletim", how="left", suffixes=("", "_logradouro")) \
                 .merge(coordenadas, on="numero_boletim", how="left", suffixes=("", "_coordenada"))

In [58]:
print(f"Observations: {df.shape[0]}, Variables: {df.shape[1]}")
df.head(2)

Observations: 456750, Variables: 59


,numero_boletim,tipo_acidente,desc_tipo_acidente,cod_tempo,desc_tempo,cod_pavimento,pavimento,cod_regional,desc_regional,origem_boletim,...,tipo_logradouro,nome_logradouro,tipo_bairro,descricao_tipo_bairro,Nº_imovel,Bairro,Data_logradouro,Hora_logradouro,longitude,latitude
0,2019026821139001,H01002,ABALROAMENTO COM VITIMA,0,NAO INFORMADO,0,NAO INFORMADO,25,PAMPULHA,POLICIA MILITAR,...,RUA,ANTONIO ALEIXO,B,BAIRRO,0,Serrano,2019-06-07,13:38:00,NaN,NaN
1,2019026821139001,H01002,ABALROAMENTO COM VITIMA,0,NAO INFORMADO,0,NAO INFORMADO,25,PAMPULHA,POLICIA MILITAR,...,RUA,DA BAHIA,B,BAIRRO,0,Centro,2019-06-07,13:38:00,NaN,NaN


In [59]:
col = df.iloc[:, 53]

print("Strings:")
print(col[col.apply(type) == str].dropna().unique()[:5])

print("\nInteiros:")
print(col[col.apply(type) == int].dropna().unique()[:5])

print("\nFloats:")
print(col[col.apply(type) == float].dropna().unique()[:5])

Strings:
['O' '8729?' '16KM' 'L' '14KM']

Inteiros:
[0 332 7300 542 1584]

Floats:
[1.3 267.01 0.2 7.5 24.5]


In [60]:
df.iloc[:, 53] = df.iloc[:, 53].fillna("").astype(str)

In [61]:
df.to_csv("../data/processed/accidents.csv", index=False)